In [1]:
import numpy as np
import matplotlib.pyplot as plt
import h5py
import scipy
from PIL import Image
from scipy import ndimage
import math

import tensorflow as tf

from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Embedding, LSTM, Dense, Bidirectional
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam

%matplotlib inline

In [2]:
tokenizer =Tokenizer()

In [ ]:
data = open('NLP_Text/data/cbt_train.txt').read()

corpus = data.lower().split("\n")

tokenizer.fit_on_texts(corpus)
total_words = len(tokenizer.word_index) + 1

print(tokenizer.word_index)
print(total_words)

In [4]:
input_sequences=[]

for line in corpus:
    token_list=tokenizer.texts_to_sequences([line])[0]
    for i in range(1, len(token_list)):
        n_gram_sequence = token_list[:i+1]
        input_sequences.append(n_gram_sequence)

max_sequence_len = max([len(x) for x in input_sequences])
input_sequences = np.array(pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre'))

xs, labels = input_sequences[:,:-1],input_sequences[:,-1]
ys = tf.keras.utils.to_categorical(labels, num_classes=total_words)

In [5]:
print(tokenizer.word_index['from'])
print(tokenizer.word_index['fairest'])

In [6]:
print(xs[32])


In [7]:
print(ys[32])


In [8]:
print(tokenizer.word_index)


In [ ]:
model=Sequential()
model.add(Embedding(total_words, 100, input_length=max_sequence_len-1))
model.add(Bidirectional(LSTM(150)))
model.add(Dense(total_words, activation='softmax'))
adam = Adam(lr=0.001)
model.compile(loss='categorical_crossentropy', optimizer=adam, metrics=['accuracy'])
#earlystop = EarlyStopping(monitor='val_loss', min_delta=0, patience=5, verbose=0, mode='auto')
history = model.fit(xs, ys, epochs=45, verbose=1)
print(model)

In [ ]:
import matplotlib.pyplot as plt


def plot_graphs(history, string):
  plt.plot(history.history[string])
  plt.xlabel("Epochs")
  plt.ylabel(string)
  plt.show()

plot_graphs(history, 'accuracy')


In [ ]:
seed_text = "twinkle twinkle little star\n"
next_words = 100
i=1  
for _ in range(next_words):
    token_list = tokenizer.texts_to_sequences([seed_text])[0]
    token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
    predicted = model.predict_classes(token_list, verbose=0)
    output_word = ""
    for word, index in tokenizer.word_index.items():
        if index == predicted:
            output_word = word
            break
    if i%6==0:
        seed_text += "\n" + output_word
    else:
        seed_text += " " + output_word
        
    i =i+1
        
print(seed_text)

In [ ]:
import gzip, json

In [ ]:
for line in gzip.open("DataSets/gutenberg-poetry-v001.ndjson.gz"):
    allLines.append(json.loads(line.strip()))

In [ ]:
import random

In [ ]:
random.sample(allLines,8)

In [ ]:
from collections import namedtuple


In [ ]:
with open('Poems/gutenberg-poetry-v001.ndjson') as json_file:
    data= json.loads(json_file)